In [ ]:
# Get AWS Account ID
import boto3
sts = boto3.client('sts')
try:
    account_id = sts.get_caller_identity()['Account']
    print(f"AWS Account ID: {account_id}")
except Exception as e:
    print(f"Error getting account ID: {e}")

# Get current region
session = boto3.Session()
current_region = session.region_name
print(f"Current region: {current_region}")

In [ ]:
import boto3
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"UserID: {identity['UserId']}")
print(f"Account: {identity['Account']}")
print(f"ARN: {identity['Arn']}")

In [ ]:
%pip install ipywidgets

In [ ]:
import boto3
import json
import ipywidgets as widgets
from IPython.display import display

def check_user_dependencies():
    username_widget = widgets.Text(
        description="Old Username:",
        placeholder="Enter old username"
    )
    submit_button = widgets.Button(description="Submit")
    output = widgets.Output()
    
    display(username_widget, submit_button, output)
    
    def on_submit(button):
        with output:
            output.clear_output()
            old_username = username_widget.value
            iam = boto3.client('iam')
            
            print(f"\nChecking dependencies for user: {old_username}")
            print("-" * 50)
            
            # Get user policies
            print("\n1. Attached Policies:")
            try:
                policies = iam.list_attached_user_policies(UserName=old_username)
                for policy in policies['AttachedPolicies']:
                    print(f"- {policy['PolicyName']}")
            except Exception as e:
                print(f"Error checking policies: {e}")
            
            # Get user groups
            print("\n2. Group Memberships:")
            try:
                groups = iam.list_groups_for_user(UserName=old_username)
                for group in groups['Groups']:
                    print(f"- {group['GroupName']}")
            except Exception as e:
                print(f"Error checking groups: {e}")
            
            # Get access keys
            print("\n3. Access Keys:")
            try:
                keys = iam.list_access_keys(UserName=old_username)
                for key in keys['AccessKeyMetadata']:
                    print(f"- {key['AccessKeyId']} (Status: {key['Status']})")
            except Exception as e:
                print(f"Error checking access keys: {e}")

    submit_button.on_click(on_submit)

check_user_dependencies()

# Also show current identity for verification
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print("\nCurrent Identity:")
print(f"UserID: {identity['UserId']}")
print(f"Account: {identity['Account']}")
print(f"ARN: {identity['Arn']}")

In [ ]:
import boto3
import json
from datetime import datetime
from tabulate import tabulate
from typing import Dict, List, Any
from concurrent.futures import ThreadPoolExecutor

def get_bedrock_regions() -> List[str]:
    """
    Returns list of AWS regions where Bedrock is likely to be available.
    Update this list as AWS expands Bedrock availability.
    """
    return [
        "us-east-1",  # N. Virginia
        "us-east-2",  # Ohio
        "us-west-1",  # N. California
        "us-west-2",  # Oregon
        "ap-northeast-1",  # Tokyo
        "ap-southeast-1",  # Singapore
        "ap-southeast-2",  # Sydney
        "eu-central-1",  # Frankfurt
        "eu-west-1"   # Ireland
    ]

def check_region_setup(region: str) -> Dict[str, Any]:
    """Check Bedrock setup for a specific region"""
    results = {
        "region": region,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "models": [],
        "inference_profiles": [],
        "access_status": [],
        "quotas": [],
        "errors": []
    }

    try:
        # Initialize regional clients
        bedrock = boto3.client('bedrock', region_name=region)
        bedrock_runtime = boto3.client('bedrock-runtime', region_name=region)
        
        # 1. List Foundation Models
        try:
            models = bedrock.list_foundation_models()
            for model in models['modelSummaries']:
                # Filter for specific providers if needed
                if model.get('providerName') in ['Anthropic', 'Amazon', 'Meta']:
                    details = {
                        'modelId': model['modelId'],
                        'provider': model.get('providerName', 'Unknown'),
                        'status': model.get('modelLifecycle', {}).get('status', 'Unknown'),
                        'inferenceTypes': model.get('inferenceTypesSupported', []),
                        'inputModalities': model.get('inputModalities', []),
                        'outputModalities': model.get('outputModalities', []),
                        'customizationsSupported': model.get('customizationsSupported', [])
                    }
                    results['models'].append(details)
        except Exception as e:
            results['errors'].append(f"Models error: {str(e)}")

        # 2. Check Cross-Region Inference Profiles
        try:
            profiles = bedrock.list_inference_profiles()
            results['inference_profiles'] = profiles.get('inferenceProfiles', [])
        except Exception as e:
            results['errors'].append(f"Profiles error: {str(e)}")

        # 3. Check Model Access
        try:
            access = bedrock.list_model_access()
            results['access_status'] = access.get('modelAccess', [])
        except Exception as e:
            results['errors'].append(f"Access error: {str(e)}")

        # 4. Check Quotas
        try:
            quotas = boto3.client('service-quotas', region_name=region)
            quota_response = quotas.list_service_quotas(ServiceCode='bedrock')
            for quota in quota_response['Quotas']:
                if any(term in quota['QuotaName'].lower() for term in 
                      ['invoke', 'model', 'request', 'token', 'concurrent']):
                    results['quotas'].append({
                        'name': quota['QuotaName'],
                        'value': quota['Value'],
                        'adjustable': quota['Adjustable'],
                        'unit': quota.get('Unit', 'Unknown')
                    })
        except Exception as e:
            results['errors'].append(f"Quotas error: {str(e)}")

    except Exception as e:
        results['errors'].append(f"Region setup error: {str(e)}")

    return results

def print_region_results(results: Dict[str, Any]) -> None:
    """Pretty print results for a region"""
    region = results['region']
    print(f"\n{'='*20} Region: {region} {'='*20}")
    
    if results['models']:
        print("\n=== Available Models ===")
        headers = ['Model ID', 'Provider', 'Status', 'Inference Types']
        table_data = [[
            m['modelId'],
            m['provider'],
            m['status'],
            ','.join(m['inferenceTypes'])
        ] for m in results['models']]
        print(tabulate(table_data, headers=headers, tablefmt='grid'))

    if results['inference_profiles']:
        print("\n=== Inference Profiles ===")
        for profile in results['inference_profiles']:
            print(f"\nProfile: {profile.get('inferenceProfileId', 'Unknown')}")
            print(f"Regions: {profile.get('awsRegions', [])}")

    if results['quotas']:
        print("\n=== Quotas ===")
        headers = ['Name', 'Value', 'Adjustable', 'Unit']
        table_data = [[
            q['name'],
            q['value'],
            q['adjustable'],
            q['unit']
        ] for q in results['quotas']]
        print(tabulate(table_data, headers=headers, tablefmt='grid'))

    if results['errors']:
        print("\n=== Errors ===")
        for error in results['errors']:
            print(f"- {error}")

def check_bedrock_all_regions(save_to_file: bool = True) -> Dict[str, Any]:
    """Check Bedrock setup across all regions"""
    all_results = {
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "regions": {}
    }

    regions = get_bedrock_regions()
    
    # Use ThreadPoolExecutor for parallel execution
    with ThreadPoolExecutor(max_workers=len(regions)) as executor:
        future_to_region = {executor.submit(check_region_setup, region): region 
                          for region in regions}
        
        for future in future_to_region:
            region = future_to_region[future]
            try:
                results = future.result()
                all_results["regions"][region] = results
                print_region_results(results)
            except Exception as e:
                print(f"\nError checking region {region}: {e}")

    if save_to_file:
        filename = f"bedrock_setup_check_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        with open(filename, 'w') as f:
            json.dump(all_results, f, indent=2, default=str)
        print(f"\nResults saved to {filename}")

    return all_results

# Usage example:
if __name__ == "__main__":
    print("Checking Bedrock setup across all regions...")
    print(f"Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    results = check_bedrock_all_regions()
    
    print(f"\nEnd time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
import boto3

bedrock = boto3.client('bedrock', region_name='us-east-1')
try:
    print("Checking model access...")
    response = bedrock.get_foundation_model(
        modelIdentifier="anthropic.claude-3-sonnet-20240229-v1:0"
    )
    print("Model status:", response)
except Exception as e:
    print(f"Error checking model: {e}")


In [ ]:
import os
from typing import Any
from dotenv import load_dotenv
from anthropic import AnthropicBedrock

load_dotenv()

client = AnthropicBedrock(
    aws_access_key=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    aws_region="us-east-1",
)

try:
    print("Attempting with Claude Instant...")
    response = client.messages.create(
        model="anthropic.claude-instant-v1",  # Try the simplest model
        max_tokens=256,
        messages=[
            {
                "role": "user",
                "content": "Hello"
            }
        ],
    )
    print("Response:", response.content[0].text)
except Exception as e:
    print(f"Error: {e}")

In [ ]:
import boto3
import os
import time
from anthropic import AnthropicBedrock
from dotenv import load_dotenv

load_dotenv()

# First cell - just setup
client = AnthropicBedrock(
    aws_access_key=os.environ.get("AWS_ACCESS_KEY_ID"),
    aws_secret_key=os.environ.get("AWS_SECRET_ACCESS_KEY"),
    aws_region="us-east-1",
)

In [ ]:
# Second cell - wait a while
print("Waiting 2 minutes for rate limits to reset...")
time.sleep(120)  # 2 minute wait

In [ ]:
# Third cell - try the request
try:
    response = client.messages.create(
        model="anthropic.claude-instant-v1",  # Starting with simplest model
        max_tokens=256,
        messages=[
            {
                "role": "user",
                "content": "Hello"
            }
        ],
    )
    print("Success!")
    print("Response:", response.content[0].text)
except Exception as e:
    print(f"Error: {e}")